# First thing to do, make a copy of this notebook under File->Make a Copy with some intelligent naming convention.

---

# Set your observing times here.

## NB: This will be a broader window that you'll actually use. The final output will give you orbit-by-orbit pointing information. So you can use a subset of the orbits that are chosen here.

In [11]:
from datetime import datetime
import numpy as np
tstart = '2017-10-05T10:00:00'
tend = '2017-10-06T10:00:00'
 
# Turn these into datetime objects

tstart = datetime.strptime(tstart, '%Y-%m-%dT%H:%M:%S')
tend = datetime.strptime(tend, '%Y-%m-%dT%H:%M:%S')                

---
# Step 1, get the nominal RA/Dec Position for the Moon at the start.

## We use this to determine what the occultation times are.

In [12]:
from skyfield.api import Loader
from astropy.time import Time
import astropy.units as u

load = Loader('../../data')

ts = load.timescale()
planets = load('de436.bsp')

astro_time = Time(tstart)
t = ts.from_astropy(astro_time)
moon, earth = planets['moon'], planets['earth']
astrometric = earth.at(t).observe(moon)
ra, dec, distance = astrometric.radec()


occstring = "./occ {0:0.4f} {1:0.4f} Latest_TLE.txt {2}:{3}:00:00:00 {4}:{5}:00:00:00 lunar_{3}_{5}.occ".format(
      ra.to(u.deg).value, dec.to(u.deg).value,
      tstart.timetuple().tm_year, tstart.timetuple().tm_yday, 
      tend.timetuple().tm_year, tend.timetuple().tm_yday)


outfile = '../orbit_engine/lunar_{0}to{1}.sh'.format(tstart.timetuple().tm_yday, tend.timetuple().tm_yday)
print(outfile)

f = open(outfile, 'w')
f.write(occstring)
f.close()


import os
import stat

st = os.stat(outfile)
os.chmod(outfile, st.st_mode | stat.S_IEXEC)



[#################################] 100% deltat.preds


../orbit_engine/lunar_278to279.sh


---
# Step 2: Go run the code that figures out the unocculted periods for the RA/Dec and the date range reported above.

This works on lif. There are example shell scripts in the ../orbit_engine directory that use the version that Karl already compiled for the nuops users.

First, get the latest TLE archive:

`./get_latest_TLE.sh`

Run the script that was produced above.


# Step 3: Initialize your libraries and parse the resulting occultation file:


In [13]:
from nustar_planning import io
occfile= "../orbit_engine/lunar_{0}_{1}.occ".format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )

#print(occfile)
orbits = io.parse_occ(occfile)

# NB: The "head" command here only shows the first couple of rows. Do a "print(orbits)" to see them all.
orbits.head()


,visible,occulted
0,2017-10-05 01:27:57,2017-10-05 02:23:46
1,2017-10-05 03:04:37,2017-10-05 04:00:25
2,2017-10-05 04:41:16,2017-10-05 05:37:05
3,2017-10-05 06:17:55,2017-10-05 07:13:44
4,2017-10-05 07:54:35,2017-10-05 08:50:23


# Use SkyField to get the location of Jupiter for each orbit:

This puts the output into the provided text file. This shows the *aim* time that was used to determine the pointing. You should slew while the source is occulted.

In [14]:
from nustar_planning import lunar_planning
outfile = 'lunar_{0}_{1}_pointing.txt'.format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )
lunar_planning.position(orbits, load_path ='../../data',
                 parallax_correction=True,
                 outfile=outfile, show=True)
print("Output is stored in: {}".format(outfile))

[#################################] 100% deltat.preds


Aim Time            RA         Dec
2017-10-05T01:27:57 4.63431  -2.05128
2017-10-05T01:39:06.800000 4.59230  -2.08801
2017-10-05T01:50:16.600000 4.08563  -2.06029
2017-10-05T02:01:26.400000 3.42446  -1.96291
2017-10-05T02:12:36.200000 3.00812  -1.82937
2017-10-05T03:04:37 5.50549  -1.74366
2017-10-05T03:15:46.600000 5.47453  -1.77581
2017-10-05T03:26:56.200000 4.97498  -1.74503
2017-10-05T03:38:05.800000 4.31279  -1.64774
2017-10-05T03:49:15.400000 3.88775  -1.51725
2017-10-05T04:41:16 6.37718  -1.43525
2017-10-05T04:52:25.800000 6.35794  -1.46284
2017-10-05T05:03:35.600000 5.86558  -1.42893
2017-10-05T05:14:45.400000 5.20218  -1.33155
2017-10-05T05:25:55.200000 4.76853  -1.20399
2017-10-05T06:17:55 7.24974  -1.12619
2017-10-05T06:29:04.800000 7.24237  -1.14920
2017-10-05T06:40:14.600000 6.75767  -1.11208
2017-10-05T06:51:24.400000 6.09339  -1.01457
2017-10-05T07:02:34.200000 5.65093  -0.88993
2017-10-05T07:54:35 8.12363  -0.81663
2017-10-05T08:05:44.600000 8.12745  -0.83495
2017-10-05

In [15]:
from nustar_planning.lunar_planning import position_shift



In [16]:
# PA == 0 (default), Rmoon = 940 arcseconds (default), 6 arcminutes maximum drift (default) and
# no minimum dwell period (default)

position_shift(orbits, parallax_correction=True)

[#################################] 100% deltat.preds


In [10]:
# PA == 30, Rmoon = 930 arcseconds, 6 arcminutes maximum drift (default) and
# a minimum dwell period of 200 seconds (default):
position_shift(orbits, parallax_correction=True, Rmoon=930*u.arcsec, 
              min_dwell=200*u.s, pa = 30*u.deg)


[#################################] 100% deltat.preds


In [ ]:
# Test PA = 0 vs PA 180
ra = 4.5574242917772985
dec = -2.349650003454101
test = SkyCoord(ra, dec, unit=u.deg)

ra = 4.5574242917772985
dec =-1.8274277812318789
test2 = SkyCoord(ra, dec, unit=u.deg)

dr = test.separation(test2)
dr

In [ ]:
# Test PA 90 vs PA 270
ra = 4.296139610238649
dec = -2.08853889234299
test = SkyCoord(ra, dec, unit=u.deg)
ra = 4.818708973315948 
dec =-2.08853889234299
test2 = SkyCoord(ra, dec, unit=u.deg)
test2.separation(test)

In [39]:
pos = position_shift(orbits, parallax_correction=True, diag=True, min_dwell=240*u.s)

[#################################] 100% deltat.preds


Start of dwell: 2017-10-05T01:22:57
End of dwell: 2017-10-05T01:27:02

Time used to aim: 2017-10-05T01:24:59.500000
Dwell Duration (sec): 245.00 
  Dwell Drift (arcmin): 8.27

Start of dwell: 2017-10-05T01:27:02
End of dwell: 2017-10-05T01:41:22

Time used to aim: 2017-10-05T01:34:12
Dwell Duration (sec): 860.00 
  Dwell Drift (arcmin): 6.11

Start of dwell: 2017-10-05T01:41:22
End of dwell: 2017-10-05T01:45:27

Time used to aim: 2017-10-05T01:43:24.500000
Dwell Duration (sec): 245.00 
  Dwell Drift (arcmin): 10.45

Start of dwell: 2017-10-05T01:45:27
End of dwell: 2017-10-05T01:49:32

Time used to aim: 2017-10-05T01:47:29.500000
Dwell Duration (sec): 245.00 
  Dwell Drift (arcmin): 13.10

Start of dwell: 2017-10-05T01:49:32
End of dwell: 2017-10-05T01:53:37

Time used to aim: 2017-10-05T01:51:34.500000
Dwell Duration (sec): 245.00 
  Dwell Drift (arcmin): 14.67

Start of dwell: 2017-10-05T01:53:37
End of dwell: 2017-10-05T01:57:42

Time used to aim: 2017-10-05T01:55:39.500000
Dwell Du